In [1]:
%matplotlib widget
import numpy as np
import os
import h5py
import subprocess

print(os.getpid())
%cd ../
import pylib.mix as mix
%cd "./jupyter-notebooks/QSVT-QSP-angles-for-matlab-c#/Hamiltonian/QSP/"

14383
/media/work/docs/codes/QuCF/scripts-py
/media/work/docs/codes/QuCF/scripts-py/jupyter-notebooks/QSVT-QSP-angles-for-matlab-c#/Hamiltonian/QSP


In [2]:
mix.reload_module(mix)

# -----------------------------------------------------------
def find_exp(number) -> int:
    base10 = np.log10(abs(number))

    temp = np.floor(base10)
    flag_neg = False
    if temp < 0:
        flag_neg = True

    return abs(temp), flag_neg
# -----------------------------------------------------------
def txt_to_hdf5(old_filename, new_filename):
    function_type_res = "QSP-ham"
    rescaling_factor_res = 1.
    parity_res = -1

    # --- read the .angles (text) file ---
    print("read angles from:\n " + old_filename)
    with open(old_filename) as f:
        param_res = float(f.readline()[:-1]) 
        eps_res   = float(f.readline()[:-1]) 
        Na        = int(f.readline()[:-1]) 
        angles = np.zeros(Na)
        for ii in range(Na):
            angles[ii] = float(f.readline())
            
    print()
    print("function type: \t\t{:s}".format(function_type_res))    
    print("rescaling factor: \t{:0.3f}".format(rescaling_factor_res))
    print("parity: \t\t{:d}".format(parity_res))
    print("parameter: \t\t{:0.3e}".format(param_res))
    print("error: \t\t\t{:0.3e}".format(eps_res))
    print("N-angles: \t\t{:d}".format(Na))
    print()       

    # --- write the .hdf5 file ---
    print("write angles to:\n " + new_filename)
    with h5py.File(new_filename, "w") as f:
        grp = f.create_group("basic")
        
        grp.create_dataset('polynomial_type',  data=function_type_res)
        grp.create_dataset('rescaling_factor', data=float(rescaling_factor_res))
        grp.create_dataset('parity',           data=parity_res)
        grp.create_dataset('eps',              data=eps_res)
        grp.create_dataset('par',              data=param_res)

        grp = f.create_group("angles")
        grp.create_dataset('QSP-ham',  data = angles)
    return

In [3]:
# ----------------------------------------------------------
# --- Specify QSP parameters ---
# ----------------------------------------------------------
t_moments = [2.236e+01]

Nt = len(t_moments)
eps_QSP_array = [1e-12] * Nt

case_format = 1

# --- Form the name of the output name ---
file_names = [None] * Nt
print("--- Chosen QSP parameters ---")
if case_format == 1:
    for it in range(Nt):
        eps_QSP = eps_QSP_array[it]
        t1 = t_moments[it]
        exp_t1, flag_neg = find_exp(t1)

        str_exp = "p"
        if flag_neg:
            str_exp = "m"

        if flag_neg:
            temp_int = int(t1 * 10**(exp_t1+3))
        else:
            temp_int = int(t1 / 10**(exp_t1-3))

        file_names[it] = "QSP_t{:d}{:s}{:d}_eps{:d}".format(
            temp_int, 
            str_exp,
            int(exp_t1),
            int(abs(np.round(np.log10(eps_QSP))))
        )

        print("t, eps: {:0.3e}, {:0.3e}".format(t1, eps_QSP))
        print("file name: {:s}".format(file_names[it]))
        print()

--- Chosen QSP parameters ---
t, eps: 2.236e+01, 1.000e-12
file name: QSP_t2236p1_eps12



In [4]:
# ------------------------------------------------
# --- Compute QSP angles ---
# ------------------------------------------------
line_code = "/media/work/docs/codes/progs/Q#F#/test-qsp/bin/Debug/netcoreapp3.1/qsp"
for it in range(Nt):
    eps_QSP = eps_QSP_array[it]
    t1 = t_moments[it]

    line_cmd = "{:s} {:0.3e} {:0.3e} {:s} {:s}".format(
        line_code, t1, eps_QSP, "./", file_names[it]
    )
    subprocess.run(line_cmd, shell = True, executable="/bin/bash")

    # from the .txt file to .hdf5 file:
    txt_to_hdf5(file_names[it], file_names[it] + ".hdf5")

    # remove the .txt file:
    line_cmd = "rm {:s}".format(file_names[it])
    subprocess.run(line_cmd, shell = True, executable="/bin/bash")

Start calculating...
Chosen tau = 22.36
Chosen eps = 1E-12
polyA = { Denominator = 281474976710656
  Numerator =
             1z^-52 + 19z^-50 + 348z^-48 + 5593z^-46 + 81344z^-44 + 1066892z^-42 + 12536678z^-40 + 130990600z^-38 + 1206314356z^-36 + 9689055506z^-34 + 67012417452z^-32 + 392777564658z^-30 + 1910994394072z^-28 + 7502511317813z^-26 + 22796710680533z^-24 + 50004400571069z^-22 + 68352521613306z^-20 + 32366944235304z^-18 + -43245263822082z^-16 + -30192174019936z^-14 + 49875475191312z^-12 + -13858871940832z^-10 + -25588252511644z^-8 + 44805115157362z^-6 + -47777564144828z^-4 + 44974108494362z^-2 + -43319993788157z^0 + 44974108494362z^2 + -47777564144828z^4 + 44805115157362z^6 + -25588252511644z^8 + -13858871940832z^10 + 49875475191312z^12 + -30192174019936z^14 + -43245263822082z^16 + 32366944235304z^18 + 68352521613306z^20 + 50004400571069z^22 + 22796710680533z^24 + 7502511317813z^26 + 1910994394072z^28 + 392777564658z^30 + 67012417452z^32 + 9689055506z^34 + 1206314356z^36 + 1309